In [2]:
import torch
from torch import nn
import os
import pickle


In [3]:
from vb import CustomDataset

In [4]:
# train_dataset = CustomDataset(train=True, target_len=2**13)
# test_dataset = CustomDataset(train=False, target_len=2**13, fixed=True)


In [5]:

# with open('train_dataset.pkl', 'wb') as f:
#     pickle.dump(train_dataset, f, pickle.HIGHEST_PROTOCOL)

# with open('test_dataset.pkl', 'wb') as f:
#     pickle.dump(test_dataset, f, pickle.HIGHEST_PROTOCOL)


In [6]:
with open('train_dataset.pkl', 'rb') as f:
    train_dataset = pickle.load(f)
with open('test_dataset.pkl', 'rb') as f:
    test_dataset = pickle.load(f)

In [7]:
from torch import nn
import torch


class CBP(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(CBP, self).__init__()
        self.conv = nn.Conv1d(in_channels, out_channels, 3, 1, padding=1)
        self.bn = nn.BatchNorm1d(out_channels)
        self.prelu = nn.PReLU()
    def forward(self, x, residual=None):
        x = self.conv(x)
        x = self.bn(x)
        if residual==None:
            x = self.prelu(x)
        else:
            x = self.prelu(x+residual)
        return x
        
class Pool(nn.Module):
    def __init__(self, in_channels):
        super(Pool, self).__init__()
        self.conv = nn.Conv1d(in_channels, in_channels, 2, 2, padding=0)
        self.bn = nn.BatchNorm1d(in_channels)
        self.prelu = nn.PReLU()
        
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.prelu(x)
        
        return x
        
        
class unPool(nn.Module):
    def __init__(self, in_channels):
        super(unPool, self).__init__()
        self.tconv = nn.ConvTranspose1d(in_channels, in_channels, 2, 2)
        self.bn = nn.BatchNorm1d(in_channels)
        self.prelu = nn.PReLU()
        
    def forward(self, x):
        x = self.tconv(x)
        x = self.bn(x)
        x = self.prelu(x)
        
        return x
          
        
        
        
        
        

class Encoder(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Encoder, self).__init__()
        
        self.residual = nn.Conv1d(in_channels, out_channels, 1)  # Skip connection
        self.cbp11 = CBP(in_channels, out_channels)
        self.cbp12 = CBP(out_channels, out_channels)
        
        self.cbp21 = CBP(out_channels, out_channels)
        self.cbp22 = CBP(out_channels, out_channels)
        
        self.pool = Pool(out_channels)
        
        
    def forward(self, x):
        residual = self.residual(x)
        x = self.cbp11(x)
        residual = self.cbp12(x, residual)
        
        x = self.cbp21(residual)
        skip = self.cbp22(x, residual)
        
        x = self.pool(skip)
        
        return x, skip

    
    
    
    
    
class Decoder(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Decoder, self).__init__()
        self.unpool = unPool(in_channels)
        self.residual = nn.Conv1d(in_channels, out_channels, 1)  # Skip connection
        
        self.cbp11 = CBP(in_channels*2, out_channels)
        self.cbp12 = CBP(out_channels, out_channels)
        
        self.cbp21 = CBP(out_channels, out_channels)
        self.cbp22 = CBP(out_channels, out_channels)
        
    def forward(self, x, skip):
        
    
        x = self.unpool(x)
        residual = self.residual(x)
        x = self.cbp11(torch.cat([x, skip], dim=1))
        residual = self.cbp12(x, residual)
    
        x = self.cbp21(residual)
        x = self.cbp22(x, residual)
        
        return x


class Feature_Encoder(nn.Module):
    def __init__(self):
        super(Feature_Encoder, self).__init__()
        # Encoder blocks
        self.encoder_block_1 = Encoder(1, 8)
        self.encoder_block_2 = Encoder(8, 16)
        self.encoder_block_3 = Encoder(16, 32)
        self.encoder_block_4 = Encoder(32, 64)
        self.encoder_block_5 = Encoder(64, 128)

        # Decoder blocks
        self.decoder_block_5 = Decoder(128, 64)
        self.decoder_block_4 = Decoder(64, 32)
        self.decoder_block_3 = Decoder(32, 16)
        self.decoder_block_2 = Decoder(16, 8)
        self.decoder_block_1 = Decoder(8, 8)
        
        self.conv1 = nn.Conv1d(8, 16, 3, 1, padding=1)  # Remove dilation
        self.conv2 = nn.Conv1d(16, 32, 3, 1, padding=1)  # Remove dilation
        self.conv3 = nn.Conv1d(32, 64, 3, 1, padding=1)
        self.conv4 = nn.Conv1d(64, 128, 3, 1, padding=1)
        self.conv7 = nn.Conv1d(128, 1, 1, 1)

        
        self.bn1 = nn.BatchNorm1d(16)
        self.bn2 = nn.BatchNorm1d(32)
        self.bn3 = nn.BatchNorm1d(64)
        self.bn4 = nn.BatchNorm1d(128)
        
        self.prelu1 = nn.PReLU()
        self.prelu2 = nn.PReLU()
        self.prelu3 = nn.PReLU()
        self.prelu4 = nn.PReLU()
        
        self.tanh = nn.Tanh()

    def forward(self, x):
        out, skip1 = self.encoder_block_1(x)
        out, skip2 = self.encoder_block_2(out)
        out, skip3 = self.encoder_block_3(out)
        out, skip4 = self.encoder_block_4(out)
        out, skip5 = self.encoder_block_5(out)
        
        out = self.decoder_block_5(out, skip5)
        out = self.decoder_block_4(out, skip4)
        out = self.decoder_block_3(out, skip3)
        out = self.decoder_block_2(out, skip2)
        out = self.decoder_block_1(out, skip1)

        out = self.conv1(out)
        out = self.bn1(out)
        out = self.prelu1(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.prelu2(out)
        out = self.conv3(out)
        out = self.bn3(out)
        out = self.prelu3(out)
        out = self.conv4(out)
        out = self.bn4(out)
        out = self.prelu4(out)

        out = self.conv7(out)

        return self.tanh(out)

model = Feature_Encoder()
sample_input = torch.randn(64, 1, 8192)
model(sample_input)
trainable_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
trainable_parameters

493327

In [9]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True, pin_memory=True,drop_last=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False, pin_memory=True,drop_last=True)
device = 'cuda'

In [10]:

from torch.utils.tensorboard import SummaryWriter

device = 'cuda'
model = Feature_Encoder().to(device)
loss_fn = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

writer = SummaryWriter('boards/0_5M_unet_tanh')
x_back = None
l_list = []
trainable_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('trainable parameters:', trainable_parameters)
for epoch in range(0,100000000000):
    t_loss = 0
    model.train()
    for clean, noisy in train_loader:
        clean, noisy = clean.to(device), noisy.to(device)
        
        pred = model(noisy)
        
        loss = loss_fn(pred, clean)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        t_loss+=loss.item()
    # l_list.append(t_loss)
    t_loss /= len(train_loader)
    
    
    
    model.eval()
    with torch.no_grad():
        t_tloss = 0
        for clean, noisy in test_loader:
            clean, noisy = clean.to(device), noisy.to(device)
            pred = model(noisy)
            loss = loss_fn(pred, clean)
            t_tloss+=loss.item()
        # l_list.append(t_tloss)
        t_tloss /= len(test_loader)
        
    writer.add_scalars('run_14h', {'train_loss':t_loss,
                                        'test_loss':t_tloss}, epoch)
    writer.flush()
    if epoch%10==1:
        print(epoch, t_loss, t_tloss)
        try:
            os.mkdir("ckpts/0_5M_unet_tanh")
        except:
            pass
        torch.save(model.state_dict(), f'ckpts/0_5M_unet_tanh/ckpt_{epoch}_.pt')

writer.close()

trainable parameters: 493327
1 0.009841566385390858 0.005455110338516533
11 0.0016458085647577214 0.0012423052685335279
21 0.0011796479100464946 0.0008039500486726562
31 0.0009145488490402285 0.0005335670939530246
41 0.0007238560095882147 0.0003706049683387391
51 0.0006308366457233206 0.00028605440820683725
61 0.0005921532632783055 0.00025599062185695704
71 0.0005588125770575263 0.00023529209526410946
81 0.0005352863906106601 0.00023545399502230188
91 0.0005290998721547011 0.0002190106145765943
101 0.0005079836222446627 0.00022240525383191803
111 0.000500962243111442 0.00020639850966593562
121 0.000490267681177809 0.00019858916251299283
131 0.00047976467823092307 0.00019644543499452993


KeyboardInterrupt: 

In [8]:
import tqdm
import librosa
import torchaudio
import numpy as np

In [11]:
with open("data/test_dataset.txt", 'r') as f:
    test_list = f.readlines()
    test_list = [x.strip() for x in test_list]

clean_path = "/workspace/nas-dataset/jis/voicebank_demand/clean_trainset_56spk_wav/"
noisy_path = "/workspace/nas-dataset/jis/voicebank_demand/noisy_trainset_56spk_wav/"
x1_path = "/workspace/nas-dataset/jis/VB_readymade/sample_enh/0_5M_unet/"
device = 'cuda'
try:
    os.mkdir(x1_path)
except:
    pass

model = model.to(device)
model.load_state_dict(torch.load('ckpts/0_5M_unet_tanh/ckpt_131_.pt'))
model.eval()
with torch.no_grad():
    for i, file in enumerate(tqdm.tqdm(test_list)):
        clean, sr1 = librosa.load(os.path.join(clean_path, file), sr=16000,res_type='kaiser_best')
        noisy, sr2 = librosa.load(os.path.join(noisy_path, file), sr=16000,res_type='kaiser_best')
        
        pad_size = 8192 - len(noisy)%8192
        padded_noisy = np.pad(noisy, (0, pad_size), 'constant', constant_values=(0, 0))
        x1 = model(torch.tensor(padded_noisy).float().view(1,1,-1).to(device)).cpu().detach().numpy().reshape(-1)[:len(clean)]
        
        torchaudio.save(os.path.join(x1_path, file), torch.tensor(x1).view(1,-1), 16000)
        
        # break

  0%|          | 0/835 [00:00<?, ?it/s]100%|██████████| 835/835 [01:27<00:00,  9.54it/s]
